# Script pour manipuler un tirage et comparer des colonnes

In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob
import itertools as it
import networkx as nx
import numpy as np

In [2]:
lexiquePrefix="MGC-160104"

with open("/Users/gilles/Box Sync/2015-Data/MGC-Vienna/"+lexiquePrefix+'-Verbes2.pkl', 'rb') as input:
    lexique = pickle.load(input)

In [49]:
nbCases=lexique.groupby(by=["lexeme","case"],as_index=False)["phono"].count()
nbFormes=nbCases.groupby(by=["lexeme"],as_index=True)[["case"]].count()
if len(nbFormes.columns)==2:
    nbFormes.columns=["lexeme","nbFormes"]
else:
    nbFormes.columns=["nbFormes"]

In [51]:
dictNbFormes=nbFormes.to_dict()["nbFormes"]

In [52]:
with open("/Users/gilles/Box Sync/2015-Data/MGC-Vienna/"+lexiquePrefix+'-Verbes2-NbForms.pkl', 'wb') as output:
    pickle.dump(dictNbFormes, output, pickle.HIGHEST_PROTOCOL)